In [1]:
import sys
import os

module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
from src.metrics import *
from settings import *
from src.data import generate

instruments = ['Guitar']
instruments_aug = ['Accordion', 'Violin', 'Piano']

# from setup_logging import setup_logging
# setup_logging()

#generate.my_run(instruments)
datasets_raw = [pd.read_pickle(os.path.join(METADATA_DIR_PROCESSED, f'data_{instrument.lower()}.pkl')) for instrument in instruments]
datasets_augmented = [pd.read_pickle(os.path.join(METADATA_DIR_AUGMENTED_RAW, f'data_{instrument.lower()}.pkl')) for instrument in instruments_aug]

2023-05-19 14:21:05.858883: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-19 14:21:05.985031: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-19 14:21:06.629821: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvrtc.so.11.0: cannot open shared object file: No such file or directory
2023-05-19 14:21:06.629896: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64]

In [2]:
print([x.shape for x in datasets_augmented[0]['spectrogram']])

[(128, 64), (128, 70), (128, 79), (128, 81), (128, 66), (128, 77), (128, 74), (128, 63), (128, 55), (128, 69), (128, 58), (128, 77), (128, 87), (128, 66), (128, 70), (128, 37), (128, 70), (128, 87), (128, 22), (128, 78), (128, 72), (128, 29), (128, 87), (128, 79), (128, 84), (128, 51), (128, 87), (128, 87), (128, 87), (128, 87), (128, 59), (128, 49), (128, 71), (128, 78), (128, 57), (128, 84), (128, 73), (128, 65), (128, 72), (128, 63), (128, 87), (128, 84), (128, 69), (128, 80), (128, 87), (128, 87), (128, 87), (128, 42), (128, 75), (128, 87), (128, 87), (128, 87), (128, 51), (128, 87), (128, 73), (128, 87), (128, 87), (128, 87), (128, 49), (128, 43), (128, 87), (128, 81), (128, 79), (128, 66), (128, 48), (128, 87), (128, 87), (128, 78), (128, 87), (128, 87), (128, 27), (128, 77), (128, 55), (128, 87), (128, 87), (128, 87), (128, 71), (128, 70), (128, 56), (128, 66), (128, 87), (128, 78), (128, 87), (128, 61), (128, 70), (128, 76), (128, 82), (128, 28), (128, 60), (128, 87), (128, 53)

In [2]:
from src.data.preprocessing import get_max_shape
max_spectrogram_size = max(map(lambda df: get_max_shape(df), datasets_raw+datasets_augmented))
max_spectrogram_size

107

In [3]:
from src.data.preprocessing import uniform_shape
uniform = lambda df: uniform_shape(df, max_spectrogram_size)

datasets_raw = list(map(uniform,datasets_raw))
datasets_augmented = list(map(uniform,datasets_augmented))
datasets_augmented = list(map(lambda df: df[['spectrogram','class_ID', 'class_name','augmentation']],datasets_augmented))
datasets_augmented = list(map(lambda df: df.reset_index(drop=True), datasets_augmented))
#datasets_augmented = list(map(lambda df: df[['spectrogram','class_ID', 'class_name','augmentation']],map(uniform,datasets_augmented)))

In [15]:
from src.data.preprocessing import all_equal
#equal_shape = 

(128, 107)

In [6]:
import random
import keras
import os, glob
import logging
import librosa, librosa.display

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from keras.models import model_from_json
from keras import backend as K

from src.metrics import *
from settings import *
from src.data import generate
from src.processing import *
from src.model import CNN
from src.data.preprocessing import get_most_shape
from setup_logging import setup_logging

setup_logging()
logger = logging.getLogger('src.train')

train_datas = []
test_datas = []

for dataset in datasets_raw:
    train_data, test_data = train_test_split(dataset, augmented=False, split_ratio=0.65)
    train_datas.append(train_data)
    test_datas.append(test_data)

for dataset in datasets_augmented:
    train_data, test_data = train_test_split(dataset, augmented=False, split_ratio=0.65)
    train_datas.append(train_data)
    test_datas.append(test_data)

train_data = pd.concat(train_datas)
test_data = pd.concat(test_datas)

train_data = train_data.sample(frac=1).reset_index(drop=True)
test_data = test_data.sample(frac=1).reset_index(drop=True)

most_shape = get_most_shape(train_data)

logger.info(f"Number of train samples: {len(train_data)}")
logger.info(f"Number of test samples: {len(test_data)}")
# most_shape = get_most_shape(dataset)
#train_data, test_data = train_test_split(dataset, augmented=augmented, split_ratio=0.65)

X_train, y_train = features_target_split(train_data)
X_test, y_test = features_target_split(test_data)

# Reshape for CNN input
X_train, X_test = reshape_feature_CNN(X_train, size=max_spectrogram_size), reshape_feature_CNN(X_test, size=max_spectrogram_size)

# Preserve y_test values
y_test_values = y_test.copy()

# One-Hot encoding for classes
y_train, y_test = one_hot_encode(y_train), one_hot_encode(y_test)

# Instance of CNN model
cnn = CNN(most_shape)
logger.info(str(cnn))

cnn.train(X_train, y_train, X_test, y_test, batch_size=16)
cnn.evaluate(X_train, y_train, X_test, y_test)

if tf.__version__ != '1.8.0':
    predict_x=cnn.model.predict(X_test)
    predictions = np.argmax(predict_x,axis=1)
else:    
    predictions = cnn.model.predict_classes(X_test)
conf_matrix=confusion_matrix(y_test_values, predictions, labels=range(10))
logger.info('Confusion Matrix for classes {}:\n{}'.format(CLASSES, conf_matrix))
#cnn.save_model(name="model_all_data_augment_1")

src.processing - INFO - Start train test split with split ratio: 0.65
src.processing - INFO - Number of training samples is 1300
src.processing - INFO - Number of testing samples is 700
src.processing - INFO - Train test split completed
src.processing - INFO - Start train test split with split ratio: 0.65
src.processing - INFO - Number of training samples is 585
src.processing - INFO - Number of testing samples is 315
src.processing - INFO - Train test split completed
src.processing - INFO - Start train test split with split ratio: 0.65
src.processing - INFO - Number of training samples is 585
src.processing - INFO - Number of testing samples is 315
src.processing - INFO - Train test split completed
src.processing - INFO - Start train test split with split ratio: 0.65
src.processing - INFO - Number of training samples is 585
src.processing - INFO - Number of testing samples is 315
src.processing - INFO - Train test split completed
The most frequent shape is (128, 107)
src.train - INFO 

In [7]:
test_instruments = instruments + instruments_aug
for test_data, instrument in zip(test_datas, test_instruments):
    X_test = test_data['spectrogram']
    X_test = np.array([x.reshape( (128, max_spectrogram_size, 1) ) for x in X_test])
    y_test = test_data['class_ID']

    y_test_values=y_test
    y_test = np.array(keras.utils.to_categorical(y_test, 10))

    score = cnn.model.evaluate(X_test,y_test)
    print(f'Test score for instrument: {instrument}')
    print('\tTest loss:', score[0])
    print('\tTest accuracy:', score[1])
    print('\tTest precision:', score[2])
    print('\tTest recall:', score[3])
    print('\tTest f1-score:', score[4])

22/22 [==============================] - 0s 7ms/step - loss: 0.4463 - accuracy: 0.8957 - precision: 0.9109 - recall: 0.8825 - fmeasure: 0.8961
Test score for instrument: Guitar
	Test loss: 0.44629520177841187
	Test accuracy: 0.895714282989502
	Test precision: 0.910923421382904
	Test recall: 0.8825080990791321
	Test f1-score: 0.8961212038993835
10/10 [==============================] - 0s 7ms/step - loss: 0.6718 - accuracy: 0.9619 - precision: 0.9706 - recall: 0.9594 - fmeasure: 0.9649
Test score for instrument: Accordion
	Test loss: 0.6718100905418396
	Test accuracy: 0.961904764175415
	Test precision: 0.9706451296806335
	Test recall: 0.9593750238418579
	Test f1-score: 0.9648745656013489
10/10 [==============================] - 0s 7ms/step - loss: 0.1668 - accuracy: 0.9492 - precision: 0.9599 - recall: 0.9438 - fmeasure: 0.9514
Test score for instrument: Violin
	Test loss: 0.16679321229457855
	Test accuracy: 0.9492063522338867
	Test precision: 0.9599078893661499
	Test recall: 0.943750023

In [8]:
cnn.save_model(name="model_alldata_augment_batchsize16")

src.model - INFO - Saving model
src.model - INFO - Saved model to /home/tzag/danigil/dl/guitarCR/models


In [7]:
with open(MODEL_1_JSON, "r") as json_file:
	loaded_model_json = json_file.read()

baseline_model = model_from_json(loaded_model_json)
baseline_model.load_weights(MODEL_1_H5)

baseline_model.compile(
            optimizer="Adam",
            loss="categorical_crossentropy",
            metrics=['accuracy', precision, recall, fmeasure])

for test_data, instrument in zip(test_datas, test_instruments):
    X_test = test_data['spectrogram']
    X_test = np.array([x.reshape( (128, max_spectrogram_size, 1) ) for x in X_test])
    y_test = test_data['class_ID']

    y_test_values=y_test
    y_test = np.array(keras.utils.to_categorical(y_test, 10))

    score = baseline_model.evaluate(X_test,y_test)
    print(f'Test score for instrument: {instrument}')
    # print('\tTest loss:', score[0])
    # print('\tTest accuracy:', score[1])
    # print('\tTest precision:', score[2])
    # print('\tTest recall:', score[3])
    print('\tf1-score:', score[4])

ValueError: in user code:

    File "/home/tzag/miniconda3/envs/danigil-steganalysis/lib/python3.8/site-packages/keras/engine/training.py", line 1820, in test_function  *
        return step_function(self, iterator)
    File "/home/tzag/miniconda3/envs/danigil-steganalysis/lib/python3.8/site-packages/keras/engine/training.py", line 1804, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/tzag/miniconda3/envs/danigil-steganalysis/lib/python3.8/site-packages/keras/engine/training.py", line 1792, in run_step  **
        outputs = model.test_step(data)
    File "/home/tzag/miniconda3/envs/danigil-steganalysis/lib/python3.8/site-packages/keras/engine/training.py", line 1756, in test_step
        y_pred = self(x, training=False)
    File "/home/tzag/miniconda3/envs/danigil-steganalysis/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "